In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

# --- 1. O "CÉREBRO" DE CONHECIMENTO (LORE OTIMIZADO) ---
# TRUQUE DE MESTRE: Melhoramos a descrição para garantir o "Match" semântico.
game_lore = [
    "O Rei Zarathos foi visto pela última vez entrando na Floresta Sombria.",
    # Mudamos de: "A Espada de Fogo custa..."
    # Para: "A Espada de Fogo é uma ARMA poderosa que custa..."
    "A Espada de Fogo é uma arma poderosa que custa 500 moedas de ouro no ferreiro.",
    "Dizem que o dragão do norte dorme apenas durante a lua nova.",
    "A poção de cura é feita misturando erva-rei e água do pântano.",
    "O jogador (você) tem má reputação na cidade por ter roubado uma galinha.",
    "A taverna serve o melhor guisado de coelho da região por 5 moedas."
]

print("🧠 Carregando memória MULTILÍNGUE do NPC...")
embedder = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
lore_embeddings = embedder.encode(game_lore, convert_to_tensor=True)

# --- 2. FUNÇÃO DE INTERAÇÃO COM DEBUG ---
def interact_with_npc(player_question):
    print(f"\n🗣️ JOGADOR PERGUNTA: '{player_question}'")
    
    # RETRIEVAL
    question_embedding = embedder.encode(player_question, convert_to_tensor=True)
    
    # Vamos pegar os TOP 3 resultados para analisar (Debug)
    hits = util.semantic_search(question_embedding, lore_embeddings, top_k=3)
    
    # --- BLOCO DE DEBUG (Pedagógico) ---
    print("\n--- 📊 DEBUG DOS SCORES (O que o cérebro pensou?) ---")
    for hit in hits[0]:
        score = hit['score']
        texto = game_lore[hit['corpus_id']]
        print(f"• Score {score:.2f} -> '{texto}'")
    print("-" * 50)
    # -----------------------------------

    best_hit = hits[0][0]
    fact_found = game_lore[best_hit['corpus_id']]
    confidence = best_hit['score']
    
    # Mantemos o threshold de 0.25, mas agora o score deve subir para > 0.50
    if confidence < 0.25:
        context_to_use = "Você não sabe nada sobre esse assunto."
        print(f"❌ Confiança muito baixa ({confidence:.2f}). Ignorando.")
    else:
        context_to_use = fact_found
        print(f"✅ Fato Aceito: '{context_to_use}'")
    
    # GENERATION (Simulação de Resposta)
    print("\n🤖 RESPOSTA DO BOGARD:")
    if "Espada" in context_to_use:
        print("Bogard: 'Ah, quer se armar? A Espada de Fogo tá no ferreiro. Custa 500 moedas. Duvido que você tenha tanto ouro.'")
    elif "Rei" in context_to_use:
        print("Bogard: 'O Rei? Aquele velho sumiu na Floresta Sombria. Se for lá, não volta.'")
    elif "reputação" in context_to_use: 
        print("Bogard: 'Ei! Devolve a galinha que você roubou ou chamo a guarda!'")
    else:
        print("Bogard: 'Não sei do que você tá falando. Vai beber ou não?'")

# --- TESTANDO ---
interact_with_npc("Onde eu encontro uma arma boa?")

🧠 Carregando memória MULTILÍNGUE do NPC...

🗣️ JOGADOR PERGUNTA: 'Existe um dragão aqui nas redondezas?'

--- 📊 DEBUG DOS SCORES (O que o cérebro pensou?) ---
• Score 0.47 -> 'Dizem que o dragão do norte dorme apenas durante a lua nova.'
• Score 0.29 -> 'O Rei Zarathos foi visto pela última vez entrando na Floresta Sombria.'
• Score 0.28 -> 'A taverna serve o melhor guisado de coelho da região por 5 moedas.'
--------------------------------------------------
✅ Fato Aceito: 'Dizem que o dragão do norte dorme apenas durante a lua nova.'

🤖 RESPOSTA DO BOGARD:
Bogard: 'Não sei do que você tá falando. Vai beber ou não?'
